In [1]:
import matplotlib.pyplot as plt
import numpy as np
from mkl_random import multivariate_normal
from scipy.linalg import hankel, svd
from numpy.random import randn

In [2]:
# Generate random matrices representing a linear (discrete-time) time-invariant DS
def gen_random_lti(n, m, p):
    A = randn(n, n)
    B = randn(n, m)
    C= randn(p, n)
    D = randn(p, m)
    return A, B, C, D

In [3]:
# Simulate I/O data from (A, B, C, D) for input vector u
def simulate_lti(A, B, C, D, u, x0=None):
    # Extract dimensions
    n, m = B.shape
    p = C.shape[0]
    T = u.shape[1]

    # Pre-allocate state & output vectors
    x = np.zeros((n, T+1))
    y = np.zeros((p, T))

    if x0 is not None:
        x[:, 0] = x0
    for t in range(T):
        x[:, t+1] = A @ x[:, t] + B @ u[:, t]
        y[:, t] = C @ x[:, t] + D @ u[:, t]

    return x,y

In [4]:
# Ho-Kalman algorithm
def ho_kalman(y, u, n):
    T = y.shape[1]
    L = T // 2
    H = hankel(y[:, :L], y[:, L-1:T-1])
    U, S, Vh = svd(H)
    O = U[:, :n] @ np.diag(np.sqrt(S[:n]))
    R = np.diag(np.sqrt(S[:n])) @ Vh[:n, :]
    return O, R # Observability & Reachability matrices

In [5]:
# Bandit/Rl-inspired sequential identification algorithm (Placeholder)
def naive_bandit_sysid(y, u, n, method='thompson'):
    T = y.shape[1]
    interval = 10
    estimated_As = []
    prior_mean = np.zeros((n, n))
    prior_cov = np.eye(n)

    for i in range(interval, T, interval):
        H = hankel(y[:, :i//2], y[:, i//2-1:i-1])
        U, S, Vh = svd(H)

        if method == 'thompson':
            sample = multivariate_normal(prior_mean.flatten(), prior_cov).reshape(n, n)
            A_est = sample
        elif method == 'ucb':
            alpha = 0.1
            uncertainty = np.sqrt(np.diag(prior_cov)).reshape(n, n)
            A_est = prior_mean + alpha * uncertainty
        else:
            A_est = U[:, :n] @ np.diag(S[:n]) @ Vh[:n, :]
        estimated_As.append(A_est)
    return estimated_As

In [6]:
# Simple identification performance evaluator
def evaluate(true_A, estimated_As):
    errors = [np.linalg.norm(true_A - A_est, ord='fro') for A_est in estimated_As]
    return errors

In [7]:
def plot_errors(errors_bandit, errors_kalman, label):
    intervals = np.arange(10, 10 * len(errors_bandit) + 1, 10)
    plt.figure(figsize=(10, 6))
    plt.plot(intervals, errors_bandit, label=f'{label}', marker='o')
    plt.plot(intervals, [errors_kalman]*len(intervals), label='Ho-Kalman', linestyle='--')
    plt.xlabel('Time Step')
    plt.ylabel('Frobenius Norm Error')
    plt.title(f'Identification Error: {label} vs. Ho-Kalman')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [8]:
# Example usage
# if __name__ == "__main__":
# Specify system dimensions and the length of the input sequence
n, m, p, T = 4, 2, 2, 100
A, B, C, D = gen_random_lti(n, m, p)
# Generate random input sequence
u = randn(m, T)
x, y = simulate_lti(A, B, C, D, u)

# Ho-Kalman
O, R = ho_kalman(y, u, n)
ho_kalman_err = np.linalg.norm(A - O[:, :n], ord='fro')

# Bandit-based
thompson_estimations = naive_bandit_sysid(y, u, n, 'thompson')
thompson_err = evaluate(A, thompson_estimations)

ucb_estimations = naive_bandit_sysid(y, u, n, 'ucb')
ucb_err = evaluate(A, ucb_estimations)

# Visualize evaluation results
plot_errors(thompson_err, ho_kalman_err, 'Thompson Sampling')
plot_errors(ucb_err, ho_kalman_err, 'UCB')

ValueError: operands could not be broadcast together with shapes (4,4) (100,4) 